In [ ]:
%%writefile score.R
fahrenheit_to_celsius <- function(temp_F) {
  temp_C <- (temp_F - 32) * 5 / 9
  return(temp_C)
}


- AML scoring expects python file and below is example of how you can use R function from Python using rpy2 package ( https://rpy2.github.io/doc/latest/html/introduction.html# )

In [ ]:
%%writefile score.py
import readline
import joblib
import os
import json
import numpy as np
import pandas as pd
import rpy2
import rpy2.robjects as ro
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri
from rpy2.robjects.conversion import localconverter
from inference_schema.schema_decorators import input_schema, output_schema
from inference_schema.parameter_types.numpy_parameter_type import NumpyParameterType
from azureml.core.model import Model

def init():
    global r_model_path,r_entry_script, score_function_r
    # The AZUREML_MODEL_DIR environment variable indicates
    # a directory containing the model file you registered.
    
    # r_model_path = Model.get_model_path(model_name='my-r-model')    
    r_entry_script = Model.get_model_path(model_name='my-r-script')
    
    # Defining the R script and loading the instance in Python
    r = ro.r
    r['source'](r_entry_script)
    # Loading the function we have defined in R.
    score_function_r = ro.globalenv['fahrenheit_to_celsius']

def run(raw_data):
    pd_df = pd.DataFrame(json.loads(raw_data)['data'])
    with localconverter(ro.default_converter + pandas2ri.converter):
        r_from_pd_df = ro.conversion.py2rpy(pd_df)

    # r_from_pd_df
    df_result_r = score_function_r(r_from_pd_df)

    with localconverter(ro.default_converter + pandas2ri.converter):
        pd_from_r_df = ro.conversion.rpy2py(df_result_r)

    return pd_from_r_df.to_json()


- A custom docker image is created to build, compile R and install any required R libraries. NOTE: Intial build can take upto 20 min

In [2]:
%%writefile deployment/Dockerfile
FROM mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04
RUN mkdir /usr/share/man/man1 && apt-get update && apt-get install --no-install-recommends -y openjdk-8-jdk
ENV JAVA_HOME /usr/lib/jvm/java-8-openjdk-amd64/
ENV DEBIAN_FRONTEND=noninteractive
RUN apt-get update \
  && apt-get install --yes \
    libssl-dev \
    libfuse-dev \
    python3 python3-pip \
    wget \
    openjdk-8-jdk \
    build-essential gfortran libreadline-dev libxml2-dev libcurl4-openssl-dev libpcre2-dev libbz2-dev liblzma-dev \
  && apt-get clean \
  && rm -rf /var/lib/apt/lists/* /tmp/* /var/tmp/*
# Conda Environment
ENV MINICONDA_VERSION py37_4.9.2
ENV PATH /opt/miniconda/bin:$PATH
RUN wget -qO /tmp/miniconda.sh https://repo.continuum.io/miniconda/Miniconda3-${MINICONDA_VERSION}-Linux-x86_64.sh && \
    bash /tmp/miniconda.sh -bf -p /opt/miniconda && \
    conda clean -ay && \
    rm -rf /opt/miniconda/pkgs && \
    rm /tmp/miniconda.sh && \
    find / -type d -name __pycache__ | xargs rm -rf
RUN wget -qO R-4.0.4.tar.gz https://cran.r-project.org/src/base/R-4/R-4.0.4.tar.gz && \
    tar -xvf R-4.0.4.tar.gz && \
    cd R-4.0.4 && \
    ./configure --enable-R-shlib --with-x=no --without-recommended-packages && \
    make -j4 && make install && rm -rf /R-4.0.4.tar.gz
ENV LD_LIBRARY_PATH="/usr/local/lib/R/lib:$LD_LIBRARY_PATH"
RUN ldconfig
RUN R -e "install.packages(c('dplyr'), repos = 'https://cloud.r-project.org/')"
RUN R -e "install.packages(c('conflicted'), repos = 'https://cloud.r-project.org/')"

Writing deployment/Dockerfile


- Python dependency file to read input data, pass to R function and respond back in json

In [3]:
%%writefile deployment/conda_dependencies.yml
channels:
- conda-forge
dependencies:
- python=3.7
- pip:
  - azureml-core==1.34.0
  - azureml-defaults==1.34.0
  - azureml-telemetry==1.34.0
  - azureml-train-restclients-hyperdrive==1.34.0
  - azureml-train-core==1.34.0
  - azureml-monitoring
  - joblib
  - pandas
  - tzlocal
  - rpy2==3.4.5
name: azureml_4dd1b7149337b0438e0e64ae5a60fd4e


Writing deployment/conda_dependencies.yml


- Deployment

In [ ]:
from azureml.core import Image, Workspace, Webservice, Model, Environment
from azureml.core.webservice import AksWebservice
from azureml.core.authentication import ServicePrincipalAuthentication
from azureml.core.model import InferenceConfig
from azureml.core.compute import ComputeTarget
from azureml.core.conda_dependencies import CondaDependencies

# Choose a name for your AKS cluster
aks_name = 'cpu-inference'    
aks_target = ComputeTarget(workspace=workspace, name=aks_name)

# Create a customer docker container with R
r_env = Environment("custom")
r_env.docker.enabled = True
r_env.docker.base_image = None
r_env.inferencing_stack_version='latest'
r_env.docker.base_dockerfile = "./deployment/Dockerfile"
r_env.python.conda_dependencies = CondaDependencies(conda_dependencies_file_path='./deployment/conda_dependencies.yml')

inference_config = InferenceConfig(entry_script="score.py", environment=r_env)
aks_config = AksWebservice.deploy_configuration(collect_model_data=True, enable_app_insights=True)

# # Sample r model or reference file
# azureml_r_model = Model.register(workspace=ws,
#                        model_name='my-r-model',      # Name of the registered model in your workspace.
#                        model_path='/tmp/class.rds',  # Local file to upload and register as a model.
#                        description='R Model RDS',
#                        tags={'area': 'azureml', 'type': 'databricks notebook', 'language': 'R'})
# R entry script 
azureml_r_entry_script = Model.register(workspace=ws,
                       model_name='my-r-script', # Name of the registered model in your workspace.
                       model_path='./score.R',   # Local file to upload and register as a model.
                       description='R model scoring scrpt',
                       tags={'area': 'azureml', 'type': 'databricks notebook', 'language': 'R'})

# Deploy webservice
print("Deploying web service")
aks_service = Model.deploy(workspace=ws,
                           name="r-model-f-to-c",
                           models=[azureml_r_entry_script],
                           inference_config=inference_config,
                           deployment_config=aks_config,
                           deployment_target=aks_target,
                          overwrite=True)

aks_service.wait_for_deployment(show_output = False)
